Import Libraries

In [1]:
!pip install tweet-preprocessor
#!pip install mlxtend
!pip install scipy
from scipy import stats
!pip install seaborn
import seaborn as sns
import sklearn
import numpy as np
import pandas as pd
import re
import json
import preprocessor as p
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
# Gaussian vs. Bernoulli NB
#from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

Import Dataset

In [2]:
# Je nach Dataset müssen wir dieses evtl aufteilen um Trainings- und Testdaten zu erhalten (70/30)

# training data
train = pd.read_csv("./datasets/detecting-insults-in-social-commentary/train.csv")
train_for_cv = pd.read_csv("./datasets/detecting-insults-in-social-commentary/train.csv")

# test data without labels to see how our classifiers perform on unlabeled and unseen data
#test = pd.read_csv("./datasets/detecting-insults-in-social-commentary/test.csv")

# dev dataset
dev = pd.read_csv("./datasets/detecting-insults-in-social-commentary/test_with_solutions.csv")
dev_for_cv = pd.read_csv("./datasets/detecting-insults-in-social-commentary/test_with_solutions.csv")

# test dataset with labels to calculate metrics
test = pd.read_csv("./datasets/detecting-insults-in-social-commentary/impermium_verification_labels.csv")
test_for_cv = pd.read_csv("./datasets/detecting-insults-in-social-commentary/impermium_verification_labels.csv")

Look at Dataset

In [3]:
train.head()
train.tail()

,Insult,Date,Comment
3942,1,20120502172717Z,"""you are both morons and that is never happening"""
3943,0,20120528164814Z,"""Many toolbars include spell check, like Yahoo..."
3944,0,20120620142813Z,"""@LambeauOrWrigley\xa0\xa0@K.Moss\xa0\nSioux F..."
3945,0,20120528205648Z,"""How about Felix? He is sure turning into one ..."
3946,0,20120515200734Z,"""You're all upset, defending this hipster band..."


In [4]:
dev.head()
dev.tail()

,Insult,Date,Comment,Usage
2642,0,20120612003508Z,"""Never really gave it much thought. I just fig...",PrivateTest
2643,0,20120619210456Z,"""Nadie se salva de la regla 34 xd""",PrivateTest
2644,0,20120528234613Z,"""Question: Are you a boy or a girl?""",PrivateTest
2645,1,20120619153537Z,"""Leave your email or phone number and maybe yo...",PrivateTest
2646,0,20120620000237Z,"""From the scenarios you present, I see you bel...",PrivateTest


In [5]:
test.head()
test.tail()

,id,Insult,Date,Comment,Usage
2230,2231,0,20120528100303Z,"""FUCKIN LAME COME ON WTF STOP FUCKING OVER MY ...",PrivateTest
2231,2232,1,20120531185813Z,"""YOU SHUT YOUR IGNORANT PIE HOLE YOU LITTLE IN...",PrivateTest
2232,2233,0,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,1,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest
2234,2235,0,20120531184524Z,"""Man....if you are a 3 point shooter, you must...",PrivateTest


In [6]:
# non-toxic comment
# Spalte der Label in Anführungszeichen angeben
sum(train["Insult"] == 0)

2898

In [7]:
# toxic comment
sum(train["Insult"] == 1)

1049

In [8]:
# non-toxic comment
# Spalte der Label in Anführungszeichen angeben
sum(dev["Insult"] == 0)

1954

In [9]:
# toxic comment
sum(dev["Insult"] == 1)

693

In [10]:
# non-toxic comment
# Spalte der Label in Anführungszeichen angeben
sum(test["Insult"] == 0)

1158

In [11]:
# toxic comment
sum(test["Insult"] == 1)

1077

In [12]:
train.isnull().sum()

Insult       0
Date       718
Comment      0
dtype: int64

Müssen die Daten noch aufgeräumt werden?


*   Sind special characters vorhanden? -> remove
*   also remove Punctuation (wie geht man mit "z.B." oder "U.S.A" um?)
*   tweet-preprocessor?

In [13]:
# remove special characters -> regex schnell und einfach
# set up punctuations we want to be replaced

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [14]:
# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [15]:
# clean training data
train_tweet = clean_tweets(train["Comment"])
train_tweet = pd.DataFrame(train_tweet)

# append cleaned tweets to the training data
train["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
#train.head(10)

#create separate Dataframe for creating all_data for 10-fold cross validation
train_for_cv = train
# delete column 'Date'
train_for_cv.drop('Date', inplace=True, axis=1)
train_for_cv.tail()

,Insult,Comment,clean_tweet
3942,1,"""you are both morons and that is never happening""",you are both morons and that is never happening
3943,0,"""Many toolbars include spell check, like Yahoo...",many toolbars include spell check like yahoo f...
3944,0,"""@LambeauOrWrigley\xa0\xa0@K.Moss\xa0\nSioux F...",\xa0\xa0moss\xa0\nsioux falls sd i told my boy...
3945,0,"""How about Felix? He is sure turning into one ...",how about felix he is sure turning into one he...
3946,0,"""You're all upset, defending this hipster band...",youre all upset defending this hipster bandand...


In [16]:
# clean the dev data and append the cleaned tweets to the dev data
dev_tweet = clean_tweets(dev["Comment"])
dev_tweet = pd.DataFrame(dev_tweet)
# append cleaned tweets to the training data
dev["clean_tweet"] = dev_tweet

# compare the cleaned and uncleaned tweets
#dev.tail()

#create separate Dataframe for creating all_data for 10-fold cross validation
dev_for_cv = dev
# delete column 'Date'
dev_for_cv.drop('Date', inplace=True, axis=1)
# delete column 'Usage'
dev_for_cv.drop('Usage', inplace=True, axis=1)
dev_for_cv.tail()

,Insult,Comment,clean_tweet
2642,0,"""Never really gave it much thought. I just fig...",never really gave it much thought i just figur...
2643,0,"""Nadie se salva de la regla 34 xd""",nadie se salva de la regla xd
2644,0,"""Question: Are you a boy or a girl?""",question are you a boy or a girl
2645,1,"""Leave your email or phone number and maybe yo...",leave your email or phone number and maybe you...
2646,0,"""From the scenarios you present, I see you bel...",from the scenarios you present i see you belie...


In [17]:
# clean the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["Comment"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
#test.tail()

#create separate Dataframe for creating all_data for 10-fold cross validation
test_for_cv = test
# delete column 'id'
test_for_cv.drop('id', inplace=True, axis=1)
# delete column 'Date'
test_for_cv.drop('Date', inplace=True, axis=1)
# delete column 'Usage'
test_for_cv.drop('Usage', inplace=True, axis=1)
test_for_cv.tail()

,Insult,Comment,clean_tweet
2230,0,"""FUCKIN LAME COME ON WTF STOP FUCKING OVER MY ...",fuckin lame come on wtf stop fucking over my b...
2231,1,"""YOU SHUT YOUR IGNORANT PIE HOLE YOU LITTLE IN...",you shut your ignorant pie hole you little ins...
2232,0,"""sweetie pie is looking very much like her cou...",sweetie pie is looking very much like her cous...
2233,1,"""ball4real where are you with your miami g-ayn...",ball4real where are you with your miami g ayness
2234,0,"""Man....if you are a 3 point shooter, you must...",manif you are a point shooter you must love pl...


## Prepare csv-files for merge to compute cross validation
Because my dataset is already splitted, I want to create one big dataset for performing 10-fold cross validation

In [18]:
frames = [train_for_cv, dev_for_cv, test_for_cv]
all_data = pd.concat(frames)
all_data

,Insult,Comment,clean_tweet
0,1,"""You fuck your dad.""",you fuck your dad
1,0,"""i really don't understand your point.\xa0 It ...",i really dont understand your point\xa0 it see...
2,0,"""A\\xc2\\xa0majority of Canadians can and has ...",a\\xc2\\xa0majority of canadians can and has b...
3,0,"""listen if you dont wanna get married to a man...",listen if you dont wanna get married to a man ...
4,0,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",c\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1eddn...
...,...,...,...
2230,0,"""FUCKIN LAME COME ON WTF STOP FUCKING OVER MY ...",fuckin lame come on wtf stop fucking over my b...
2231,1,"""YOU SHUT YOUR IGNORANT PIE HOLE YOU LITTLE IN...",you shut your ignorant pie hole you little ins...
2232,0,"""sweetie pie is looking very much like her cou...",sweetie pie is looking very much like her cous...
2233,1,"""ball4real where are you with your miami g-ayn...",ball4real where are you with your miami g ayness


In [19]:
# extract the labels & comments from the train data

# labels
y_train = train.Insult.values
# comments
X_train = train.clean_tweet.values

# extract the labels & comments from the dev data

# labels
y_dev = dev.Insult.values
# comments
X_dev = dev.clean_tweet.values

# extract the labels & comments from the test data

# labels
y_test = test.Insult.values
# comments
X_test = test.clean_tweet.values

Vectorize tweets using CountVectorizer

In [20]:
documents = ["This is a test document.",
             "I like trains",
             "Bla Blaa Blaaa",
            "I also just recognize words with more than 1 letter!!"]

# initializing the countvectorizer
vectorizer = CountVectorizer()

# tokenize and make the document into a matrix
document_term_matrix = vectorizer.fit_transform(documents)

# check the result
pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names())

,also,bla,blaa,blaaa,document,is,just,letter,like,more,recognize,test,than,this,trains,with,words
0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,1,0,1,1,0,1,0,0,1,1


In [21]:
# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(X_train) + list(X_dev))

# transform documents to document-term matrix
X_train_vec = vectorizer.transform(X_train)
X_dev_vec = vectorizer.transform(X_dev)
X_test_vec = vectorizer.transform(X_test)

**Support Vector Machine**

In [22]:
# classify using support vector classifier
# line below produces AttributeError: 'SVC' object has no attribute 'SVC'
#svm = svm.SVC(kernel='linear', C=1, gamma=1)
svm = SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
svm.fit(X_train_vec, y_train)

# make predictions on test data
predictions = svm.predict(X_test_vec)

# accuracy score for SVC
print("Accuracy score for SVC is: ", accuracy_score(predictions, y_test) * 100, '%')

Accuracy score for SVC is:  68.14317673378076 %


In [23]:
from sklearn.metrics import classification_report
print(classification_report(predictions,y_test))

from sklearn.metrics import f1_score
f1_score(predictions,y_test, average='macro')

              precision    recall  f1-score   support

           0       0.88      0.64      0.74      1584
           1       0.47      0.78      0.59       651

    accuracy                           0.68      2235
   macro avg       0.67      0.71      0.66      2235
weighted avg       0.76      0.68      0.70      2235



0.6641492422400519

In [24]:
# Computing cross validation for svm by splitting training data
# from: https://scikit-learn.org/stable/modules/cross_validation.html#

X  = all_data.clean_tweet.values
X_vec = vectorizer.transform(X)
X_vec = X_vec.toarray()

y = all_data.Insult.values

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#X_train.shape, y_train.shape
#X_test.shape, y_test.shape

X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)

X_train_vec = X_train_vec.toarray()
X_test_vec = X_test_vec.toarray()

clf = svm.fit(X_train_vec, y_train)
clf.score(X_test_vec, y_test)



scores_accuracy = cross_val_score(clf, X_vec, y, cv=10)
scores_accuracy



print("%0.2f accuracy with a standard deviation of %0.2f" % (scores_accuracy.mean(), scores_accuracy.std()))

0.78 accuracy with a standard deviation of 0.01


In [25]:
# computing cross-validated f1_macro
scores_f1_macro = cross_val_score(clf, X_vec, y, cv=10, scoring='f1_macro')

print("%0.2f f1_macro with a standard deviation of %0.2f" % (scores_f1_macro.mean(), scores_f1_macro.std()))

0.74 f1_macro with a standard deviation of 0.02


In [26]:
svm_scores_accuracy = []
for x in range(10):
    svm_scores_accuracy.append(scores_accuracy[x])
svm_scores_f1_macro = []
for x in range(10):
    svm_scores_f1_macro.append(scores_f1_macro[x])

In [27]:
with open('svm.all-folds-cv.f1_macro.results.json', 'w') as fp:
  json.dump(svm_scores_f1_macro, fp)
with open('svm.all-folds-cv.acc.results.json', 'w') as fp:
  json.dump(svm_scores_accuracy, fp)

In [28]:
# create Dataframe for t-test
folds = [0,1,2,3,4,5,6,7,8,9]
SVM_data = pd.DataFrame(folds, columns=['Fold'])
#NB_f1_macro = pd.DataFrame(scores_f1_macro, columns=['F1_macro'])
SVM_data['Model']='SVM'
SVM_data['F1_macro']=svm_scores_f1_macro
SVM_data['Accuracy']= svm_scores_accuracy
SVM_data

,Fold,Model,F1_macro,Accuracy
0,0,SVM,0.764046,0.798414
1,1,SVM,0.745867,0.780294
2,2,SVM,0.762821,0.796149
3,3,SVM,0.759424,0.796149
4,4,SVM,0.751291,0.787089
5,5,SVM,0.746059,0.785957
6,6,SVM,0.739618,0.780294
7,7,SVM,0.734086,0.782559
8,8,SVM,0.713673,0.755379
9,9,SVM,0.709525,0.753968


**Naïve Bayes**

In [29]:
naiveBayes = BernoulliNB()

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(X_train) + list(X_dev))

#X_train_vec = X_train_vec.toarray()
#y_train = y_train.toarray()
#X_test_vec = X_test_vec.toarray()


y_pred = naiveBayes.fit(X_train_vec, y_train).predict(X_test_vec)

print("Number of mislabeled points out of a total %d points : %d" % (X_test_vec.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 2235 points : 1066


k-fold cross validation for Naive Bayes

In [30]:
X  = all_data.clean_tweet.values
X_vec = vectorizer.transform(X)
X_vec = X_vec.toarray()

y = all_data.Insult.values

clf = naiveBayes.fit(X_train_vec, y_train)
clf.score(X_test_vec, y_test)

scores_accuracy = cross_val_score(clf, X_vec, y, cv=10)
#scores

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores_accuracy.mean(), scores_accuracy.std()))

# computing cross-validated f1_macro
scores_f1_macro = cross_val_score(clf, X_vec, y, cv=10, scoring='f1_macro')
print("%0.2f f1_macro with a standard deviation of %0.2f" % (scores_f1_macro.mean(), scores_f1_macro.std()))

0.76 accuracy with a standard deviation of 0.02
0.70 f1_macro with a standard deviation of 0.04


In [31]:
nb_scores_accuracy = []
for x in range(10):
    nb_scores_accuracy.append(scores_accuracy[x])
nb_scores_f1_macro = []
for x in range(10):
    nb_scores_f1_macro.append(scores_f1_macro[x])

In [32]:
scores_f1_macro
with open('nb.all-folds-cv.f1_macro.results.json', 'w') as fp:
  json.dump(nb_scores_f1_macro, fp)
with open('nb.all-folds-cv.acc.results.json', 'w') as fp:
  json.dump(nb_scores_accuracy, fp)

In [33]:
# create Dataframe for t-test
folds = [0,1,2,3,4,5,6,7,8,9]
NB_data = pd.DataFrame(folds, columns=['Fold'])
#NB_f1_macro = pd.DataFrame(scores_f1_macro, columns=['F1_macro'])
NB_data['Model']='NB'
NB_data['F1_macro']=nb_scores_f1_macro
NB_data['Accuracy']= nb_scores_accuracy
NB_data

,Fold,Model,F1_macro,Accuracy
0,0,NB,0.732420,0.776897
1,1,NB,0.738584,0.778029
2,2,NB,0.718586,0.763307
3,3,NB,0.761010,0.797282
4,4,NB,0.738191,0.788222
5,5,NB,0.712232,0.763307
6,6,NB,0.649213,0.728199
7,7,NB,0.638702,0.730464
8,8,NB,0.653027,0.728199
9,9,NB,0.689861,0.753968


**Logistic Regression**

In [34]:
logisticRegr = LogisticRegression()

logisticRegr.fit(X_train_vec, y_train)


predictions = logisticRegr.predict(X_test_vec)


# Measuring Model Performance
print(classification_report(predictions,y_test))


f1_score(predictions,y_test, average='macro')

              precision    recall  f1-score   support

           0       0.92      0.63      0.75      1706
           1       0.41      0.83      0.55       529

    accuracy                           0.68      2235
   macro avg       0.67      0.73      0.65      2235
weighted avg       0.80      0.68      0.70      2235



0.6472267926838602

k-fold cross validation for Logistic Regression

In [35]:
X  = all_data.clean_tweet.values
X_vec = vectorizer.transform(X)
X_vec = X_vec.toarray()

y = all_data.Insult.values

clf = logisticRegr.fit(X_train_vec, y_train)
clf.score(X_test_vec, y_test)

scores_accuracy = cross_val_score(clf, X_vec, y, cv=10)
#scores

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores_accuracy.mean(), scores_accuracy.std()))

# computing cross-validated f1_macro
scores_f1_macro = cross_val_score(clf, X_vec, y, cv=10, scoring='f1_macro')
print("%0.2f f1_macro with a standard deviation of %0.2f" % (scores_f1_macro.mean(), scores_f1_macro.std()))

0.80 accuracy with a standard deviation of 0.02
0.75 f1_macro with a standard deviation of 0.03


In [36]:
lr_scores_accuracy = []
for x in range(10):
    lr_scores_accuracy.append(scores_accuracy[x])
lr_scores_f1_macro = []
for x in range(10):
    lr_scores_f1_macro.append(scores_f1_macro[x])

In [37]:
with open('lr.all-folds-cv.f1_macro.results.json', 'w') as fp:
  json.dump(lr_scores_f1_macro, fp)
with open('lr.all-folds-cv.acc.results.json', 'w') as fp:
  json.dump(lr_scores_accuracy, fp)

In [38]:
# create Dataframe for t-test
folds = [0,1,2,3,4,5,6,7,8,9]
LogisticRegression_data = pd.DataFrame(folds, columns=['Fold'])
#NB_f1_macro = pd.DataFrame(scores_f1_macro, columns=['F1_macro'])
LogisticRegression_data['Model']='LogisticRegression'
LogisticRegression_data['F1_macro']=lr_scores_f1_macro
LogisticRegression_data['Accuracy']= lr_scores_accuracy
LogisticRegression_data

,Fold,Model,F1_macro,Accuracy
0,0,LogisticRegression,0.771229,0.814270
1,1,LogisticRegression,0.766739,0.807475
2,2,LogisticRegression,0.789175,0.825595
3,3,LogisticRegression,0.776299,0.821065
4,4,LogisticRegression,0.762890,0.806342
5,5,LogisticRegression,0.770112,0.813137
6,6,LogisticRegression,0.740467,0.796149
7,7,LogisticRegression,0.743066,0.799547
8,8,LogisticRegression,0.708808,0.763307
9,9,LogisticRegression,0.706425,0.765306


Comparing Logistic Regression & Bernoulli Naive Bayes

In [39]:
print(stats.ttest_ind(NB_data['F1_macro'], LogisticRegression_data['F1_macro']))
p = stats.ttest_ind(NB_data['F1_macro'], LogisticRegression_data['F1_macro']).pvalue
if p <= 0.05:
	print('Difference between mean performance is probably real')
else:
	print('Algorithms probably have the same performance')

Ttest_indResult(statistic=-3.08912642352205, pvalue=0.006328010687660841)
Difference between mean performance is probably real


In [40]:
print(stats.ttest_ind(NB_data['Accuracy'], LogisticRegression_data['Accuracy']))
p = stats.ttest_ind(NB_data['Accuracy'], LogisticRegression_data['Accuracy']).pvalue
if p <= 0.05:
	print('Difference between mean performance is probably real')
else:
	print('Algorithms probably have the same performance')

Ttest_indResult(statistic=-3.8617845165089433, pvalue=0.0011427048986234005)
Difference between mean performance is probably real


Comparing Logistic Regression & SVM

In [ ]:
print(stats.ttest_ind(SVM_data['Accuracy'], LogisticRegression_data['Accuracy']))
p = stats.ttest_ind(SVM_data['Accuracy'], LogisticRegression_data['Accuracy']).pvalue
if p <= 0.05:
	print('Difference between mean performance is probably real')
else:
	print('Algorithms probably have the same performance')

In [ ]:
print(stats.ttest_ind(SVM_data['F1_macro'], LogisticRegression_data['F1_macro']))
p = stats.ttest_ind(SVM_data['F1_macro'], LogisticRegression_data['F1_macro']).pvalue
if p <= 0.05:
	print('Difference between mean performance is probably real')
else:
	print('Algorithms probably have the same performance')

Concatenating Dataframes for plotting

In [ ]:
frames = [SVM_data, NB_data, LogisticRegression_data]

result = pd.concat(frames)

In [ ]:
result

In [ ]:
#baseline_no_resampling = baseline_data[baseline_data['resampled']==False]
nb = result[result['Model'] == 'NB']
lr = result[result['Model'] == 'LogisticRegression']
svm1 = result[result['Model'] == 'SVM']

In [ ]:
sns.set(style='darkgrid')
model_plot = sns.catplot(x="Model", y = "Accuracy", 
                         hue="Model", 
                         kind="bar", 
                         data=result,
                         height=7,
                         capsize=.1)
model_plot.set_axis_labels("Model", "Accuracy")

In [ ]:
sns.set(style='darkgrid')
model_plot = sns.catplot(x="Model", y = "F1_macro", 
                         hue="Model", 
                         kind="bar", 
                         data=result,
                         height=7,
                         capsize=.1)
model_plot.set_axis_labels("Model", "F1_macro")